[View in Colaboratory](https://colab.research.google.com/github/m-murphy/sketch-to-diagram/blob/master/generate_model.ipynb)

In [0]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import urllib.request

!rm -r data
!mkdir data

classes = ['circle', 'square', 'line', 'star', 'hexagon', 'octagon', 'triangle']

In [0]:

def download():
  base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
  for c in classes:
    url = f"{base}{c}.npy"
    print(url)
    urllib.request.urlretrieve(url, f'data/{c}.npy')
    
def load_data(root, vfold_ratio=0.2, max_items_per_class=10000):
  all_files = glob.glob(os.path.join(root, "*.npy"))
  
  x = np.empty([0, 784])
  y = np.empty([0])
  class_names = []
  
  for idx, file in enumerate(all_files):
    data = np.load(file)
    data = data[0: max_items_per_class, :]
    labels = np.full(data.shape[0], idx)
    
    x = np.concatenate((x, data), axis=0)
    y = np.append(y, labels)
    
    class_name, ext = os.path.splitext(os.path.basename(file))
    class_names.append(class_name)
    
  data = None
  labels = None
  
  permutation = np.random.permutation(y.shape[0])
  x = x[permutation, :]
  y = y[permutation]
  
  vfold_size = int(x.shape[0]/100 * (vfold_ratio * 100))
  
  x_test = x[0 : vfold_size, :]
  y_test = y[0 : vfold_size]
  
  x_train = x[vfold_size : x.shape[0], :]
  y_train = y[vfold_size : y.shape[0]]
  return x_train, y_train, x_test, y_test, class_names


In [0]:
download()

In [0]:
x_train, y_train, x_test, y_test, class_names = load_data('data')
num_classes = len(class_names)
image_size = 28

In [0]:
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline

idx = randint(0, len(x_train))
plt.imshow(x_train[idx].reshape(28, 28))
print(class_names[int(y_train[idx].item())])

In [0]:
x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(7, activation='softmax')) 
# Train model
adam = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['top_k_categorical_accuracy'])
print(model.summary())

In [0]:
model.fit(x = x_train, y = y_train, validation_split=0.1, batch_size = 256, verbose=2, epochs=5)

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))


In [0]:
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline  
idx = randint(0, len(x_test))
img = x_test[idx]
plt.imshow(img.squeeze()) 
pred = model.predict(np.expand_dims(img, axis=0))[0]
ind = (-pred).argsort()[:5]
latex = [class_names[x] for x in ind]
print(latex)

In [0]:
with open('class_names.txt', 'w') as file_handler:
    for item in class_names:
        file_handler.write("{}\n".format(item))

In [0]:
!pip install tensorflowjs

In [0]:
model.save('keras.h5')

!rm -r model
!mkdir model
!tensorflowjs_converter --input_format keras keras.h5 model/

!cp class_names.txt model/class_names.txt
!cp keras.h5 model/keras.h5
!zip -r model.zip model

In [0]:
from google.colab import files
files.download('model.zip')